## Watson NLP Example for Text Extensions for Pandas

This demo shows how to use the `watson` module from Text Extension for Pandas to 
process a Watson NLP response from the IBM cloud into Pandas DataFrames for analysis.
The responses should be in the form of decoded JSON Python and the following features
will be processed into DataFrames:

* entities
* keywords
* relations
* semantic_roles
* syntax with sentences and tokens

To properly authenticate with IBM Cloud, please set the environment variable
`IBM_AUTH_KEY` with your correct apikey to make requests to 
`ibm_watson.NaturalLanguageUnderstandingV1`.

In [1]:
# INITIALIZATION BOILERPLATE

# The Jupyter kernel for this notebook usually starts up inside the notebooks
# directory, but the text_extensions_for_pandas package code is in the parent
# directory. Add that parent directory to the front of the Python include path.
import sys
if (sys.path[0] != ".."):
    sys.path[0] = ".."

import json
import os
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, CategoriesOptions, ConceptsOptions, EmotionOptions, EntitiesOptions, KeywordsOptions, \
    MetadataOptions, RelationsOptions, SemanticRolesOptions, SentimentOptions, SyntaxOptions, SyntaxOptionsTokens
from text_extensions_for_pandas.io.watson import watson_nlu_parse_response

In [2]:
# Retrieve the APIKEY for authentication
apikey = os.environ.get("IBM_AUTH_KEY")
if apikey is None:
    raise ValueError("Expected apikey in the environment variable 'IBM_AUTH_KEY'")

In [3]:
# Initialize the authenticator for making requests
authenticator = IAMAuthenticator(apikey)
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2019-07-12',
    authenticator=authenticator
)

natural_language_understanding.set_service_url('https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/21b9b875-4ddb-46ad-bb22-d78747622ca7')

In [4]:
# Make the request
response = natural_language_understanding.analyze(
    url='https://raw.githubusercontent.com/CODAIT/text-extensions-for-pandas/master/resources/holy_grail.txt',
    features=Features(
        #categories=CategoriesOptions(limit=3), 
        #concepts=ConceptsOptions(limit=3), 
        #emotion=EmotionOptions(targets=['grail']),
        entities=EntitiesOptions(sentiment=True,limit=3),
        keywords=KeywordsOptions(sentiment=True,emotion=True,limit=3),
        #metadata=MetadataOptions(),
        relations=RelationsOptions(),
        semantic_roles=SemanticRolesOptions(limit=3),
        #sentiment=SentimentOptions(targets=['Arthur']),
        syntax=SyntaxOptions(sentences=True, tokens=SyntaxOptionsTokens(lemma=True, part_of_speech=True))  # Experimental
    )).get_result()

In [ ]:
# View response as JSON
print(json.dumps(response, indent=2))

In [5]:
# Get the response as processed Pandas DataFrames
dfs = watson_nlu_parse_response(response)

In [6]:
# Created DataFrames from the response
dfs.keys()

dict_keys(['entities', 'keywords', 'relations', 'semantic_roles', 'syntax'])

In [7]:
dfs['keywords']

,count,emotion.anger,emotion.disgust,emotion.fear,emotion.joy,emotion.sadness,relevance,sentiment.label,sentiment.score,text
0,1,0.071927,0.031335,0.058051,0.691404,0.175057,0.746411,neutral,0.000000,legend of King Arthur
1,1,0.021033,0.095661,0.016340,0.810654,0.046902,0.642571,positive,0.835873,Sir Lancelot
2,1,0.112061,0.033299,0.043658,0.747356,0.091490,0.642235,neutral,0.000000,King Arthur


In [8]:
dfs['entities']

,confidence,count,relevance,sentiment.label,sentiment.mixed,sentiment.score,text,type
0,1.000000,12,0.956097,negative,1,-0.312834,Arthur,Person
1,1.000000,5,0.678523,positive,None,0.835873,Lancelot,Person
2,0.977538,2,0.644313,neutral,None,0.000000,Monty Python,Person


In [9]:
dfs['relations']

,arguments.entities.disambiguation.subtype,score,sentence,type,arguments.0.entities.text,arguments.1.entities.text,arguments.0.entities.type,arguments.1.entities.type,arguments.0.location,arguments.1.location,arguments.0.text,arguments.1.text
0,"[[None], [None]]",0.462615,Monty Python and the Holy Grail is a 1975 Brit...,timeOf,1975,comedy,Date,TitleWork,"[37, 41]","[57, 61]",1975,film
1,"[[None], [None]]",0.339446,"Arthur leads the men to Camelot, but upon furt...",locatedAt,men,Camelot,Person,GeopoliticalEntity,"[1506, 1509]","[1513, 1520]",men,Camelot
2,"[[None], [None]]",0.604304,"As they turn away, God (an image of W. G. Grac...",affectedBy,their,speaks,Person,EventCommunication,"[1699, 1703]","[1689, 1695]",them,speaks
3,"[[None], [None]]",0.304596,Searching the land for clues to the Grail's lo...,locatedAt,Grail,location,Organization,Location,"[1794, 1799]","[1802, 1810]",Grail,location
4,"[[None], [[Country]]]",0.895035,Searching the land for clues to the Grail's lo...,employedBy,soldiers,French,Person,GeopoliticalEntity,"[1872, 1880]","[1865, 1871]",soldiers,French
5,"[[None], [[Country]]]",0.903545,Arthur and Bedevere eventually reach the Castl...,employedBy,soldiers,French,Person,GeopoliticalEntity,"[4952, 4960]","[4945, 4951]",soldiers,French
6,"[[None], [None]]",0.945371,Searching the land for clues to the Grail's lo...,agentOf,soldiers,claim,Person,EventCommunication,"[1881, 1884]","[1885, 1890]",who,claim
7,"[[None], [None]]",0.943395,A modern-day historian filming a documentary d...,agentOf,Grail,investigation,Organization,EventLegal,"[2455, 2461]","[2462, 2475]",police,investigation
8,"[[None], [None]]",0.600505,Sir Robin avoids a fight with a Three-Headed K...,agentOf,Robin,arguing,Person,EventCommunication,"[2656, 2661]","[2740, 2747]",Robin,arguing
9,"[[None], [None]]",0.542254,"Lancelot, after receiving an arrow-shot note f...",locatedAt,Lancelot,castle,Person,Facility,"[2894, 2902]","[3038, 3044]",Lancelot,castle


In [10]:
dfs['syntax']

,lemma,part_of_speech,char_span,token_span,sentence
0,None,PROPN,"[0, 5): 'Monty'","[0, 5): 'Monty'","[0, 273): 'Monty Python and the Holy Grail is ..."
1,python,PROPN,"[6, 12): 'Python'","[6, 12): 'Python'","[0, 273): 'Monty Python and the Holy Grail is ..."
2,and,CCONJ,"[13, 16): 'and'","[13, 16): 'and'","[0, 273): 'Monty Python and the Holy Grail is ..."
3,the,DET,"[17, 20): 'the'","[17, 20): 'the'","[0, 273): 'Monty Python and the Holy Grail is ..."
4,None,PROPN,"[21, 25): 'Holy'","[21, 25): 'Holy'","[0, 273): 'Monty Python and the Holy Grail is ..."
...,...,...,...,...,...
1076,officer,NOUN,"[5306, 5314): 'officers'","[5306, 5314): 'officers'","[5275, 5335): 'The movie ends with one of the ..."
1077,break,VERB,"[5315, 5323): 'breaking'","[5315, 5323): 'breaking'","[5275, 5335): 'The movie ends with one of the ..."
1078,the,DET,"[5324, 5327): 'the'","[5324, 5327): 'the'","[5275, 5335): 'The movie ends with one of the ..."
1079,camera,NOUN,"[5328, 5334): 'camera'","[5328, 5334): 'camera'","[5275, 5335): 'The movie ends with one of the ..."
